In [ ]:
!pip install --upgrade google-cloud-vision

In [ ]:
!pip install google-cloud-vision

In [7]:
import os
from google.cloud import vision_v1p2beta1 as vision

# Establece la variable de entorno para la autenticación
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './banded-access-402321-750cab17d500.json'

def async_detect_document(gcs_source_uri, gcs_destination_uri):
    # Crea un cliente para usar la API
    client = vision.ImageAnnotatorClient()

    # Configura la solicitud
    feature = vision.Feature(type_=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)
    gcs_source = vision.GcsSource(uri=gcs_source_uri)
    input_config = vision.InputConfig(gcs_source=gcs_source, mime_type='application/pdf')
    gcs_destination = vision.GcsDestination(uri=gcs_destination_uri)
    output_config = vision.OutputConfig(gcs_destination=gcs_destination, batch_size=1)

    async_request = vision.AsyncAnnotateFileRequest(
        features=[feature], input_config=input_config, output_config=output_config)

    operation = client.async_batch_annotate_files(requests=[async_request])

    print("Operación iniciada, esperando a que se complete...")

    # Espera a que la operación termine
    try:
        response = operation.result()
        print("Operación completada.")
        return response
    except Exception as e:
        print(f"Error: {e}")
        if operation.done():
            print("La operación ha terminado.")
        else:
            print("La operación aún no se ha completado.")


# Llama a la función con tus propios argumentos
async_detect_document('gs://bucket_db_region/oge.pdf', 'gs://bucket_db_region/transformado/')


Operación iniciada, esperando a que se complete...
Error: Could not convert Any to AsyncBatchAnnotateFilesResponse
La operación ha terminado.


In [14]:
!pip install google-cloud-vision==3.4.4

In [28]:
!pip install pdf2image

In [32]:
!sudo apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.2 [186 kB]
Fetched 186 kB in 0s (670 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 120875 

In [33]:
import io
import os

from google.cloud import vision
from pdf2image import convert_from_path

In [34]:
def ocr_pdf(pdf_path):
    """Realiza reconocimiento ORC de un PDF.

    Args:
        pdf_path: Ruta del archivo PDF.

    Returns:
        Una lista de cadenas con el texto reconocido.
    """

    # Inicializa la API de Vision.
    client = vision.ImageAnnotatorClient()

    # Convierte las páginas del PDF en imágenes.
    images = convert_from_path(pdf_path)

    for i, image in enumerate(images):
        # Convierte la imagen a bytes.
        with io.BytesIO() as output:
            image.save(output, format="PNG")
            image_bytes = output.getvalue()

        # Crea un objeto Image para la API de Vision.
        vision_image = vision.Image(content=image_bytes)

        # Realiza el reconocimiento ORC.
        response = client.text_detection(image=vision_image)

        # Verifica si se encontró algún texto.
        if response.text_annotations:
            # Guarda la respuesta en un archivo de texto.
            with open(f"output_{i}.txt", "w") as f:
                f.write(response.text_annotations[0].description)
        else:
          pass
            # print(f"No se encontró texto en la imagen {i}.")

    return response



In [35]:
pdf_path = "./oge.pdf"
ocr_pdf(pdf_path)

text_annotations {
  locale: "es"
  description: "6)\nProponer mejoras al sistema de gesti\303\263n acad\303\251mica de la UNASAM para su adecuada\nimplementaci\303\263n.\n7) Cumplir otras funciones que le asigne el Director de la OGE, en el \303\241mbito de su competencia.\nB. LINEAS DE AUTORIDAD Y RESPONSABILIDAD:\n1) Depende Jer\303\241rquicamente del DPCA\n2)\nEs responsable del proceso de funcionamiento y mantenimiento del Sistema de la\nUNASAM.\n3)\nEs responsable de cumplir en forma eficiente y oportuna las funciones se\303\261aladas en el\nreglamento de organizaci\303\263n y funciones, el manual de organizaci\303\263n y funciones y dem\303\241s\ndisposiciones legales vigentes que norman su actividad.\n4) Es responsable de los bienes, equipos y materiales que le son asignados para el\ncumplimiento de sus funciones\nREQUISITOS MINIMOS:\nTitulo Profesional relacionado con las labores del cargo.\nExperiencia en la administraci\303\263n de software manejador de bases de datos.\nAmpl